# Plot all products

Plot all products from the cleaned 50k data set for use with the interactive heatmap

In [ ]:
import pathlib
import re
import sys

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import rdMolDraw2D

sys.path.append(str(pathlib.Path().resolve().parents[1]))
from src.util.db_utils import SynFermDatabaseConnection

In [ ]:
# connect to DB
con = SynFermDatabaseConnection()

In [ ]:
res = con.con.execute("SELECT e.id, e.initiator_long, e.monomer_long, e.terminator_long, e.product_A_smiles FROM experiments e WHERE e.exp_nr BETWEEN 4 AND 29 AND (e.valid NOT LIKE '%ERROR%' OR e.valid IS NULL);").fetchall()

columns = [
    "experiment_id", 
    "I_long", 
    "M_long", 
    "T_long", 
    "product_A_smiles", 
]
df = pd.DataFrame(res, columns=columns)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
template = Chem.RWMol(Chem.MolFromSmarts("[#6]-C(=O)-[NH1]-C(-[*])-C(-[*])-c1:n:[n,c]:c:s:1"))

AllChem.Compute2DCoords(template, canonOrient=False, sampleSeed=50, nSample=200)
template

In [ ]:
# n.b. we use the SE-ethers to get a nice orientation for the molecule
template = Chem.RWMol(Chem.MolFromSmarts("[#6]-C(=O)-[NH1]-C(-[Se]1)-C(-[Se]-C-C-1)-c1:n:[n,c]:c:s:1"))

AllChem.Compute2DCoords(template, canonOrient=True, sampleSeed=0, nSample=1)
template

In [ ]:
for atom in template.GetAtoms():
    print(atom.GetIdx(), atom.GetSymbol())

In [ ]:
# delete the placeholder atoms
template.RemoveAtom(9)
template.RemoveAtom(8)
template.RemoveAtom(7)
template.RemoveAtom(5)

In [ ]:
template

In [ ]:
for i, row in df.iterrows():
    m = Chem.MolFromSmiles(row["product_A_smiles"])
    _ = AllChem.GenerateDepictionMatching2DStructure(m, template, allowRGroups=True)

    d = rdMolDraw2D.MolDraw2DSVG(300, 300)
    d.DrawMolecule(m)
    d.FinishDrawing()
    save_dir = pathlib.Path("product_images")
    save_dir.mkdir(exist_ok=True)
    with open(save_dir / f"{row['I_long']}_{row['M_long']}_{row['T_long']}.svg", "w") as f:
        f.write(re.sub('<rect.*</rect>\n', '', d.GetDrawingText()))  # remove background with regex